# Model UNet

**Objectif:** le but de ce notebook est d'expliquer le modèle UNet et ses différentes méthodes. 

### Root Variables 

In [7]:
import os 

In [8]:
root = '/home/ign.fr/ttea/Code_IGN/AerialImageDataset'
train_dir = os.path.join(root,'train/images')
gt_dir = os.path.join(root,'train/gt')
test_dir = os.path.join(root,'test/images')

In [9]:
import sys 

In [10]:
sys.path.insert(0, '/home/ign.fr/ttea/stage_segmentation_2021/Code')

### Import Libraries 

In [11]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import pandas as pd 
from prettytable import PrettyTable

In [12]:
from dataloader.dataloader import InriaDataset
from model.model import UNet
from train import train, eval, train_full

In [13]:
var= pd.read_json('variables.json')

### Inria Dataset

Définition de la partie train et validation du jeu de données. 

In [14]:
tile_size = (250,250)
train_dataset = InriaDataset(var['variables']['root'],tile_size,'train',None,False,1)
val_dataset = InriaDataset(var['variables']['root'],tile_size,'validation',None,False,1)

In [15]:
print(train_dataset[0][0].size())

torch.Size([3, 250, 250])


## U-Net Model

### Convolution Block 

La fonction conv_bloc va prendre en entrée le nombre de canal en entrée et le nombre de canal en sortie. Elle va retourner un block de deux opérations de convolutions 3x3 chacune suivi de la fonction d'activation ReLU.  

In [16]:
# Double Conv2D
def conv_block(in_channel, out_channel):
    """
    in_channel : number of input channel, int 
    out_channel : number of output channel, int
    
    Returns : Conv Block of 2x Conv2D with ReLU 
    """
    
    conv = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, kernel_size=3,padding=1),
        nn.ReLU(inplace= True),
        nn.Conv2d(out_channel, out_channel, kernel_size=3,padding=1),
        nn.ReLU(inplace= True),
    )
    return conv

### Crop 

La fonction crop est une fonction de recadrage, son objectif est de recadrer les tenseurs pour qu'ils soient de même taille car certains peuvent perdre des pixels au niveau des bordures dues aux opérations de convolutions. 

La fonction crop prend en entrée 2 tenseurs:

- Tenseur base 
- Tenseur à recadrer

Pour le recadrage, on se basera sur la taille du tenseur de base et on va ainsi recadrer l'autre tenseur.

On va mesurer l'écart entre les tailles des 2 tenseurs (delta), puis on distinguera 2 cas :

- pair : Si la taille du tenseur à recadrer - delta  est divisible par 2
- impair : Si la taille du tenseur à recadrer - delta  n'est pas divisible par 2

On retournera ensuite le tenseur recadré. 

**Exemple :** 

**tile size : (250,250)**

On affiche la taille des tenseurs par blocs : 

**block 1:** 

- u6 torch.Size([1, 128, 30, 30])
- c4 torch.Size([1, 128, 31, 31])

**block 2:**
- u7 torch.Size([1, 64, 60, 60])
- c3 torch.Size([1, 64, 62, 62])

**block 3:**
- u8 torch.Size([1, 32, 120, 120])
- c2 torch.Size([1, 32, 125, 125])

**block 4:**
- u9 torch.Size([1, 16, 240, 240])
- c1 torch.Size([1, 16, 250, 250])

On a donc besoin de recadrer chaque tenseur.

Pour le block 4:
target_tensor_size = 240
tensor_size =  250 
delta = 10 
d = delta // 2 = 5

On voit que 240- 10 % 2 = 0

On est dans le cas pair : 
- On va recadrer le tenseur en ne prenant que tenseur[:,:, d:240 -d , d: 240-d]

In [17]:
def crop(target_tensor, tensor): # x,c
    """
    target_tensor : target the tensor to crop  
    tensor: tensor 
    
    Returns : tensor cropped by half left side image concatenate with right side image
    
    """
       
    target_size = target_tensor.size()[2] 
    tensor_size = tensor.size()[2]        
    delta = tensor_size - target_size     
    delta = delta // 2                    
    
    if (tensor_size - 2*delta)%2 == 0:
      tens = tensor[:, :, delta:tensor_size- delta , delta:tensor_size-delta]

    elif (tensor_size -2*delta)%2 ==1:
      tens = tensor[:, :, delta:tensor_size- delta -1  , delta:tensor_size-delta -1]
    return tens

### UNet

![title](../img/Unet.png)

Le modèle UNet est de type encodeur-décodeur. 

Dans la première partie, on voit que l'encodeur suit une architecture typique d'un réseau de neurones convolutif. Le réseau consiste en une application répétée de 2 convolutions 3x3 chacune suivi de la fonction d'activation ReLU & d'une opération de Maxpooling (pour le downsampling). A chaque étape de sous-échantillonnage, on double le nombre de canaux.

Dans la seconde partie, chaque étape dans le décodeur consiste à un suréchantillonnage (upsampling), de la carte de caractéristiques suivi d'une convolution 2x2. Cela aura pour but de diviser par 2 le nombre de canaux. On va ensuite faire une opération de concatenation avec la feature map rognée par rapport à l'encodeur et une opération 3x3 de convolution suivi d'une ReLU. 

Ensuite, le recadrage est nécessaire à cause de la perte de pixels de bordure dans chaque convolution.
Au niveau de la couche finale, une opération de convolution 1x1 est utilisée pour mapper chaque vecteur d’entités à 64 composants au nombre de classes souhaité. Le réseau est donc composé de 23 couches convolutives. 

Lien de publication : https://arxiv.org/pdf/1505.04597.pdf

Pour mieux comprendre : https://towardsdatascience.com/understanding-semantic-segmentation-with-unet-6be4f42d4b47

Il est possible d'ajuster le modèle UNet pour avoir un code plus modulaire. 

In [18]:
class UNet(nn.Module):
  """
  UNet network for semantic segmentation
  """
  
  def __init__(self, n_channels, n_class, cuda = 1):
    """
    initialization function
    n_channels, int, number of input channel
    conv_width, int list, depth of the convs
    n_class = int,  the number of classes
    """
    super(UNet, self).__init__() #necessary for all classes extending the module class
    self.is_cuda = cuda
    self.n_class = n_class
    
    #-------------------------------------------------------------
    
    ## Encoder 
    
    # Conv2D (input channel, outputchannel, kernel size)
    
    self.c1 = conv_block(3,16)
    self.p1 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.c2 = conv_block(16,32)
    self.p2 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    self.c3 = conv_block(32,64)
    self.p3 = nn.MaxPool2d(kernel_size=2, stride=2)  
    
    self.c4 = conv_block(64,128)
    self.p4 = nn.MaxPool2d(kernel_size=2, stride=2)      
    
    self.c5 = conv_block(128,256)

    #--------------------------------------------------------------

    ## Decoder 
    
    
    # Transpose & UpSampling Convblock   
    self.t6 = nn.ConvTranspose2d(256,128, kernel_size= 2, stride=2)
    self.c6 = conv_block(256,128)

    self.t7 = nn.ConvTranspose2d(128,64, kernel_size=2, stride=2)
    self.c7 = conv_block(128,64)

    self.t8 = nn.ConvTranspose2d(64,32, kernel_size=2, stride=2)
    self.c8 = conv_block(64,32)

    self.t9 = nn.ConvTranspose2d(32,16, kernel_size=2, stride=2)
    self.c9 = conv_block(32,16)
    
    # Final Classifyer layer 
    self.outputs = nn.Conv2d(16, n_class, kernel_size= 1)
    
    #weight initialization

    self.c1[0].apply(self.init_weights)
    self.c2[0].apply(self.init_weights)
    self.c3[0].apply(self.init_weights)
    self.c4[0].apply(self.init_weights)
    self.c5[0].apply(self.init_weights)
    self.c6[0].apply(self.init_weights)
    self.c7[0].apply(self.init_weights)
    self.c8[0].apply(self.init_weights)
    self.c9[0].apply(self.init_weights)
    
    if cuda: #put the model on the GPU memory
      self.cuda()
    
  def init_weights(self,layer): #gaussian init for the conv layers
    nn.init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
    
  def forward(self, input):
    """
    the function called to run inference
    """  
    if self.is_cuda: #put data on GPU
        input = input.cuda()

    # Encoder (Left Side)
    c1=self.c1(input)
    p1=self.p1(c1)
    
    c2=self.c2(p1)
    p2=self.p2(c2)
    
    c3=self.c3(p2)
    p3=self.p3(c3)
    
    c4=self.c4(p3)
    p4=self.p4(c4)
    
    c5=self.c5(p4)    
    
    list_encoder =[c1,p1,c2,p2,c3,p3,c4,p4,c5]

    # Decoder (Right Side)
    u6=self.t6(c5)
    y4 = crop(u6,c4)
    concat4 = torch.cat([u6,y4],1)
    x6=self.c6(concat4)
    
    u7=self.t7(x6)
    y3 = crop(u7,c3)
    x7=self.c7(torch.cat([u7,y3],1))
    
    u8=self.t8(x7)
    y2 = crop(u8,c2)
    x8=self.c8(torch.cat([u8,y2],1))
    
    u9=self.t9(x8)
    y1=crop(u9,c1)
    
    x9=self.c9(torch.cat([u9,y1],1))
    
    # Final Output Layer
    out = self.outputs(x9)
    
    return out

### Test du modèle UNet 

In [19]:
img, mask = train_dataset[42]
unet = UNet(4,2)
pred = unet(img[None,:,:,:]) #the None indicate a batch dimension of 4 N,C,W,H
print('pred', pred)
print('output:',pred.shape)

pred tensor([[[[ 3.2014,  1.7763,  2.2627,  ...,  3.0320,  0.8912,  1.5148],
          [ 1.8558,  0.6278,  1.0343,  ...,  1.0752, -0.1118,  0.1783],
          [ 2.4056,  1.1724,  0.8346,  ...,  1.6552, -0.5554,  0.7397],
          ...,
          [ 5.2199,  1.9324,  2.3053,  ...,  2.4124, -1.9416,  1.1826],
          [ 4.4953,  1.7722,  1.5551,  ...,  1.7151, -2.0182,  0.4434],
          [ 2.9212,  3.2966,  2.9514,  ...,  3.6673,  1.6915,  1.9052]],

         [[ 1.4095,  0.8170,  1.0640,  ...,  1.2230,  1.3785,  1.3817],
          [ 0.9638, -0.3274, -0.2210,  ..., -0.2285,  0.6141,  0.9847],
          [ 0.3233, -0.8271, -0.4823,  ..., -0.8649, -0.3472,  1.2299],
          ...,
          [ 0.5056, -1.1974, -0.9169,  ..., -0.7821,  0.2010,  1.1125],
          [ 1.7131,  1.0656,  0.7498,  ...,  1.0461,  1.2171,  1.9487],
          [-0.0276,  1.3139,  1.4973,  ...,  1.8490,  0.7624,  1.3999]]]],
       device='cuda:0', grad_fn=<AddBackward0>)
output: torch.Size([1, 2, 240, 240])


### Nombre de Paramètres UNet

Nous allons calculer le nombre de paramètres pour l'encodeur et décodeur dans UNet, ainsi que son nombre total de paramètre. 

In [20]:
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params,table

In [21]:
total_unet, table_unet = count_parameters(unet)

+----------------+------------+
|    Modules     | Parameters |
+----------------+------------+
|  c1.0.weight   |    432     |
|   c1.0.bias    |     16     |
|  c1.2.weight   |    2304    |
|   c1.2.bias    |     16     |
|  c2.0.weight   |    4608    |
|   c2.0.bias    |     32     |
|  c2.2.weight   |    9216    |
|   c2.2.bias    |     32     |
|  c3.0.weight   |   18432    |
|   c3.0.bias    |     64     |
|  c3.2.weight   |   36864    |
|   c3.2.bias    |     64     |
|  c4.0.weight   |   73728    |
|   c4.0.bias    |    128     |
|  c4.2.weight   |   147456   |
|   c4.2.bias    |    128     |
|  c5.0.weight   |   294912   |
|   c5.0.bias    |    256     |
|  c5.2.weight   |   589824   |
|   c5.2.bias    |    256     |
|   t6.weight    |   131072   |
|    t6.bias     |    128     |
|  c6.0.weight   |   294912   |
|   c6.0.bias    |    128     |
|  c6.2.weight   |   147456   |
|   c6.2.bias    |    128     |
|   t7.weight    |   32768    |
|    t7.bias     |     64     |
|  c7.0.

### Calcul du nombre de paramètres 

Pour le calcul des paramètres sur chaque convolution, on définit la formule suivante : 

Conv = (width * height * nombre de filtres dans la couche précédente) * nombre de filtres dans la couche actuelle

Dans notre cas (width & height) sont définis dans la kernel size et le nombre de filtres sont les entrées et sorties des canaux dans la conv2d. 

Par exemple C1 =  Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

On va calculer : $(3*3*3)*16 = 432 $

<p style="text-align: center;"> <b>Nombre de paramètres pour UNet </b></p>

| Number  | Modules     | In/Out Channel | # Parameters  |
| --------|-----|---------------|-------|
| 1 |Conv1.1 (f=3) |(3,16) | 448 |
| 2 |Conv1.2 (f=3) |(16,16) | 2320 |
| 3 |Conv2.1 (f=3) |(16,32) | 4640 |
| 4 |Conv2.2 (f=3) |(32,32) | 9248 |
| 5 |Conv3.1 (f=3) |(32,64)| 18464 |
| 6 |Conv3.2 (f=3) |(64,64)| 36928 |
| 7 |Conv4.1 (f=3) |(64,128)| 73856 |
| 8 |Conv4.2 (f=3) |(128,128)| 147584|
| 9 |Conv5.1 (f=3) |(128,256)| 295168|
| 10 |Conv5.2 (f=3) |(256,256)| 590080|

**Nombre de paramètres pour l'encodeur UNet**

In [22]:
unet_encode_param = [432,16,2304,16,4608,32,9216,32,18432,64,36864,64,73728,128,147456,128,294912,256,589824,256]

In [23]:
print("Le nombre de paramètre pour l'encodeur unet est",sum(unet_encode_param))

Le nombre de paramètre pour l'encodeur unet est 1178768


<p style="text-align: center;"> <b>Nombre de paramètres pour UNet </b></p>

| Number  | Modules     | In/Out Channel | # Parameters  |
| --------|-----|---------------|-------|
| 1 |ConvT6 (f=3) |(256,128) | 131200 |
| 2 |Conv6.1 (f=3) |(256,128) | 295040 |
| 3 |Conv6.2 (f=3) |(128,128) | 147584 |
| 4 |ConvT7 (f=3) |(128,64) | 32832 |
| 5 |Conv7.1 (f=3) |(128,64)| 73792 |
| 6 |Conv7.2 (f=3) |(64,64)| 36928 |
| 7 |ConvT8 (f=3) |(64,32)| 8224 |
| 8 |Conv8.1 (f=3) |(64,32)| 18464|
| 9 |Conv8.2 (f=3) |(32,32)| 9248|
| 10 |ConvT9 (f=3) |(32,16)| 2064|
| 11 |Conv9.1 (f=3) |(32,16)|4624|
| 12 |Conv9.2 (f=3) |(16,16)|2320|
| 13 |Output(f=3) |(16,2)| 34|

**Nombre de paramètres pour le décodeur unet**

In [24]:
unet_decode_param = [131072,128,294912,128,147456,128,32768,64,73728,64,36864,64,8192,32,18432,32,9216,32,2048,16,4608,16,2304,16,32,2]

In [25]:
print("Le nombre de paramètre pour le décodeur unet est",sum(unet_decode_param))

Le nombre de paramètre pour le décodeur unet est 762354


### Arguments & Hyperparamètres 

In [26]:
hparam = {
    'lr':0.0001,
    'n_epoch':5,
    'n_epoch_test':int(5),
    'n_class':1,
    'batch_size':8,
    'n_channel':3,
    'conv_width':[16,32,64,128,256,128,64,32,16],
}

In [27]:
tile_size = (512,512)

weights = [0.5, 1.0]
class_weights = torch.FloatTensor(weights).cuda()

args = {
    'nn_loss':nn.BCEWithLogitsLoss(reduction="mean"),
    #'nn_loss':nn.CrossEntropyLoss(weight = class_weights,reduction="mean"),
    #'nn_loss':BinaryDiceLoss,
     'loss_name': 'BinaryCrossentropy',
    # 'loss_name': 'Crossentropy',
    #'loss_name':'BinaryDiceLoss',
    'threshold':0.5,
    'cuda':1,
    'class_names':['None','Batiment'],
    'save_model':False,
    'save_model_name':"unet_test_8_1.pth",
    'train_dataset':InriaDataset(var['variables']['root'],tile_size,'train',None,False,1),
    'val_dataset':InriaDataset(var['variables']['root'],tile_size,'validation',None,False,1),
}

### Entraînement du Modèle UNet

In [29]:
model = UNet(hparam['n_channel'], hparam['n_class'], cuda=args['cuda'])
trained_model, metric_train, metric_test = train_full(args, model,hparam['lr'],hparam['n_epoch'],
                                    hparam['n_epoch_test'],hparam['batch_size'],hparam['n_class'],
                                    hparam['n_channel'])

Total number of parameters: 1941105


  0%|                                                                                                         …

None : 87.64%  |  Batiment : 34.56%
Epoch   0 -> Train Overall Accuracy: 88.40% Train mIoU : 61.10% Train Loss: 0.2866
None : 87.64%  |  Batiment : 34.56%


  0%|                                                                                                         …

None : 90.58%  |  Batiment : 52.16%
Epoch   1 -> Train Overall Accuracy: 91.46% Train mIoU : 71.37% Train Loss: 0.2061
None : 90.58%  |  Batiment : 52.16%


...

None : 92.24%  |  Batiment : 61.02%
Epoch   3 -> Train Overall Accuracy: 93.08% Train mIoU : 76.63% Train Loss: 0.1693
None : 92.24%  |  Batiment : 61.02%


  0%|                                                                                                         …

None : 92.73%  |  Batiment : 63.64%
Epoch   4 -> Train Overall Accuracy: 93.55% Train mIoU : 78.19% Train Loss: 0.1585
None : 92.73%  |  Batiment : 63.64%


  0%|                                                                                                         …

None : 93.27%  |  Batiment : 63.67%
Test Overall Accuracy: 93.98% Test mIoU : 78.47%  Test Loss: 0.1479
None : 93.27%  |  Batiment : 63.67%
